# Test registration on velodyne scans

In [ ]:
%matplotlib widget
import numpy as np
import os
import time
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from copy import deepcopy
import open3d as o3d

import planeslam.io as io
from planeslam.scan import velo_pc_to_scan
from planeslam.general import NED_to_ENU, trajectory_plot_trace, pc_plot_trace
from planeslam.geometry.util import quat_to_R
from planeslam.registration import get_correspondences

os.environ['KMP_DUPLICATE_LIB_OK']='True'

%load_ext autoreload
%autoreload 2

In [ ]:
np.set_printoptions(suppress=True)

In [ ]:
# Read in point cloud data
pcpath = os.path.join(os.getcwd(),'..', '..', 'data', 'velodyne', '8_12_2022', 'flightroom', 'run_1', 'pcs')
PCs = []
for i in range(len(os.listdir(pcpath))):  # ignore first 200 and last 200 frames
    filename = pcpath+'\pc_'+str(i)+'.npy'
    PC = np.load(filename)
    PCs.append(PC)

In [ ]:
# Read in pose data
posepath = os.path.join(os.getcwd(),'..', '..', 'data', 'velodyne', '8_12_2022', 'flightroom', 'run_1', 'poses')
poses = []
for i in range(len(os.listdir(pcpath))):  # ignore first 200 and last 200 frames
    filename = posepath+'\pose_'+str(i)+'.npy'
    pose = np.load(filename)
    poses.append(pose)

In [ ]:
# Extract scans
num_scans = len(PCs)
# scans = num_scans * [None]
# scans_transformed = num_scans * [None]
select_scans = np.arange(0, num_scans, 10)
scans = len(select_scans) * [None]
for i, s in enumerate(select_scans):
    scans[i] = velo_pc_to_scan(PCs[s])
    scans[i].transform(quat_to_R(poses[s][3:]), poses[s][:3])

In [ ]:
# Plot 2 scans
source_idx = 1
target_idx = 0
source = scans[source_idx]
target = scans[target_idx]

fig = make_subplots(rows=1, cols=2, specs=[[{'type': 'surface'}, {'type': 'surface'}]])

for t in source.plot_trace(show_normals=True):
    fig.add_trace(t, row=1, col=1)

for t in target.plot_trace(show_normals=True):
    fig.add_trace(t, row=1, col=2)

fig.update_layout(width=1600, height=700, scene=dict(aspectmode='data'), scene2=dict(aspectmode='data'))
fig.show()

correspondences = get_correspondences(source, target)
print(correspondences)

In [ ]:
# Transform point clouds
combined_P = np.array([0,0,0])
for i, P in enumerate(PCs):
    P = (quat_to_R(poses[i][3:]) @ P.T).T + poses[i][:3]
    combined_P = np.vstack((combined_P, P))

In [ ]:
combined_P.shape

In [ ]:
P = o3d.geometry.PointCloud()
P.points = o3d.utility.Vector3dVector(combined_P)
o3d.visualization.draw_geometries([P])

In [ ]:
# Plot scans
fig = go.Figure()

for i, scan in enumerate(scans):
    if i % 10 == 0:
        for t in scan.plot_trace():
            fig.add_trace(t)

fig.update_layout(width=1500, height=900, scene=dict(aspectmode='data'))
fig.show()

In [ ]:
from planeslam.registration import robust_GN_register

R_abs = np.eye(3)
t_abs = np.zeros(3)
traj_est = np.zeros((num_scans, 3))
traj_est[0] = t_abs
avg_runtime = 0

for i in range(1, len(scans)):
    print("i = ", i)
    start_time = time.time()
    R_hat, t_hat = robust_GN_register(scans[i], scans[i-1])
    t_abs += (R_abs @ t_hat).flatten()
    R_abs = R_hat @ R_abs
    avg_runtime += time.time() - start_time
    traj_est[i] = t_abs

avg_runtime /= len(scans)-1
print("average registration time: ", avg_runtime)

In [ ]:
# Plot trajectories
est_traj_trace = go.Scatter3d(x=traj_est[:,0], y=traj_est[:,1], z=traj_est[:,2])
fig = go.Figure(data=[est_traj_trace])
fig.update_layout(width=1500, height=900, scene=dict(aspectmode='data'))
fig.show()